In [129]:
import numpy as np
import pandas as pd
import cv2 as cv
import os
import math

In [130]:
images=os.listdir("Data/image")
dataset=pd.DataFrame({'label':[],'R':[],'G':[],'B':[],'name':[]})


In [131]:
def extract_colors(files):
    shape=dataset.shape[0]
    for j in range(len(files)):
        img=cv.imread("Data/image/"+files[j])
        avgR = round(np.mean(img[:,:,2]),3)
        avgG = round(np.mean(img[:,:,1]),3)
        avgB = round(np.mean(img[:,:,0]),3)
        dataset.loc[j+shape]=[files[j][0],avgR,avgG,avgB,files[j]]
extract_colors(images)

,label,R,G,B,name
0,j,76.636,104.799,44.681,j28.jpg
1,j,31.138,59.272,35.637,j9.jpg
2,j,71.235,98.644,52.440,j1.jpg
3,j,77.520,86.574,64.969,j10.jpg
4,j,76.129,97.608,44.976,j11.jpg
...,...,...,...,...,...
77,s,103.468,165.880,205.668,s5.jpg
78,s,78.360,98.228,119.322,s6.jpg
79,s,49.876,81.471,100.091,s7.jpg
80,s,118.603,153.979,167.559,s8.jpg


In [132]:
class subFrame():
    """
    we create a class to extract the training samples for each class and also
    seperate our training set from our test set
    we define three options, 
    one for classes spliting and preprocessing,
    one for creatign 1vs rest naive bayes by combining the training samples of the two other classes
    and one for global training set.
    """
    def __init__(self,dataframe,column=None,class_name=None,option=1,dataframe2=None):
        if option==1:
            self.name=class_name
            self.dataframe=dataframe.loc[(dataframe[column]==class_name)]
            #self.cleans()
            #self.toNumeric()
            #self.toKG()
            #spliting sub dataframe to test set and data set
            self.testset=self.dataframe.iloc[:round((self.dataframe.shape[0]/100)*30)]
            self.dataframe=self.dataframe.iloc[round((self.dataframe.shape[0]/100)*30):]
        elif option == 0 :
            self.dataframe=pd.concat([dataframe2,dataframe],ignore_index=True)
            self.trainset=None
        elif option==2:
            self.dataframe=dataframe
            self.trainset=None
            
    def return_std(self):
        #creating a std function for each feature of dataframe
        self.std={};
        for i in list(self.dataframe)[1:4]:
            self.std[i]=round(self.dataframe[i].std(),3)
        return self.std
    def return_mean(self):
        self.mean={};
        for i in list(self.dataframe)[1:4]:
            self.mean[i]=round(self.dataframe[i].mean(),3)
        return self.mean
    def toKG(self):
        #converting body masses from g to kg to normalize the distribution of data
        self.dataframe['body_mass_g']=self.dataframe['body_mass_g'].div(1000).round(1)
    
    def cleans(self):
        """
        deleting empty rows per each subFrame
        
        """
        self.dataframe=self.dataframe[self.dataframe['body_mass_g']!='x']
        
    def toNumeric(self):
        #converting string datas to numeric datas
        for i in list(self.dataframe)[1:]:
            self.dataframe[i]=pd.to_numeric(self.dataframe[i])
            
    def returnFrame(self):
        return self.dataframe
    def returnTest(self):
        return self.testset
    def set_total(self,total):
        self.total=total
    def p_wi(self):
        return round(self.dataframe.shape[0]/self.total,6)
    def p_xi_wi(self):
        pass

In [133]:
jungle=subFrame(dataset,column='label',class_name='j')
sea=subFrame(dataset,column='label',class_name='s')

In [134]:
jungle.set_total(148)
sea.set_total(148)
trainset=subFrame(jungle.returnFrame(),option=0,dataframe2=sea.returnFrame())
testset=subFrame(jungle.returnTest(),option=0,dataframe2=sea.returnTest())
trainset=trainset.returnFrame()
testset=testset.returnFrame()

In [135]:
def likelihood(feature,class1:subFrame,label):
    std=class1.return_std()[label];
    mean=class1.return_mean()[label];
    f_x=(1/(std*math.sqrt(2*math.pi)))*(np.e**(-(((feature-mean)**2)/(2*(std**2)))))
    return f_x
def posterior (feature_vector:list,class1:subFrame,class2:subFrame):
    labels=list(dataset)[1:4]
    f_xc1=[]
    f_xc2=[]
    px_w1=1
    px_w2=1
    for i in range(len(feature_vector)):
        f_xc1.append(likelihood(feature_vector[i],class1,labels[i]))
        f_xc2.append(likelihood(feature_vector[i],class2,labels[i]))
    #print (f_xc1)
    #print(f_xc2)
    for i in range(len(feature_vector)):
        px_w1=px_w1*f_xc1[i]
        px_w2=px_w2*f_xc2[i]
    #print (px_w1)
    #print (px_w2)
    posterior_w1=(px_w1*class1.p_wi())/((px_w1*class1.p_wi())+(px_w2*class2.p_wi()))
    posterior_w2=(px_w2*class2.p_wi())/((px_w1*class1.p_wi())+(px_w2*class2.p_wi()))
    return posterior_w1

In [142]:
def naive_bayes_model(feature_vector:list,class1:subFrame,
                     class2:subFrame,):
    posteriors={}
    posteriors['j']=posterior(feature_vector,class1,class2)
    posteriors['s']=posterior(feature_vector,class2,class1)

    lst=list(posteriors)
    max=0
    choice=''
    for i in lst:
        if posteriors[i]>max:
            max=posteriors[i]
            choice=i
    return (choice,max)
features=[]
for i in range(testset.shape[0]):
    features.append(testset.iloc[i])
misses=0
print (len(features))
x11=0
x12=0
x21=0
x22=0
for i in features:
    output=naive_bayes_model([i[1],i[2],i[3]],jungle,sea)
    #print ("answer :",i[0])
    #print ("predict : ",output[0])
    #print ("========\n")
    if output[0]!=i[0]:
        misses+=1
        print ("samle missclassified : "+i[4])
        #print ("increament")
    #print (output[0],i[0])
    if i[0]=='j' and output[0]=='j':
        x11=x11+1
    elif i[0]=='j' and output[0]=='s':
        x12=x12+1
    elif i[0]=='s' and output[0]=='j':
        x21=x21+1
    elif i[0]=='s' and output[0]=='s':
        x22=x22+1
confusion_matrix=np.array([[x11,x12],[x21,x22]])
print (confusion_matrix)
print ("The accuracy", (testset.shape[0]-misses)/testset.shape[0])
print ("The recall is :",confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][0]))
print ("The percision is :",confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[0][1]))
#print (sea.returnFrame().shape,jungle.returnFrame().shape)
#print (testset)

25
samle missclassified : s13.jpg
samle missclassified : j19.jpg
[[12  1]
 [ 1 11]]
The accuracy 0.92
The recall is : 0.9230769230769231
The percision is : 0.9230769230769231
